In [ ]:
import pandas as pd
import regex as re
import json

life_df = pd.read_csv("https://raw.githubusercontent.com/AndrewZinc/Expect_Life/main/Raw_Data/Cleaning_Completed/life_expectancy_World_Bank.csv",)

In [ ]:
gdp = pd.read_csv("https://raw.githubusercontent.com/AndrewZinc/Expect_Life/main/Clean_Data/gdp_clean.csv")

In [ ]:
social = pd.read_csv("https://raw.githubusercontent.com/AndrewZinc/Expect_Life/main/Clean_Data/Cluster_Analysis_Data/country_social_security_systems-coded.csv")

In [ ]:
regions = pd.read_csv("https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv")

In [ ]:
per_cap = pd.read_csv("https://raw.githubusercontent.com/AndrewZinc/Expect_Life/Vivek_Project/Clean_Data/world_bank_gdp_per_capita_clean.csv%20-%20API_NY.GDP.PCAP.CD_DS2_en_csv_v2_4770417.csv")

In [ ]:
alc_tob = pd.read_csv("https://raw.githubusercontent.com/AndrewZinc/Expect_Life/main/Clean_Data/at_clean.csv")

In [ ]:
alc = alc_tob.loc[alc_tob['Series Name'] == 'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)']
tob = alc_tob.loc[alc_tob['Series Name'] == 'Prevalence of current tobacco use (% of adults)']

In [ ]:
master = pd.read_csv("https://raw.githubusercontent.com/AndrewZinc/Expect_Life/main/Clean_Data/master_country_list/country_profile_urls.csv")

# List of countries in master document
country_list = master['country'].to_list()

In [ ]:
# Add country code column to gdp to allow gdp figure addition to geojson file

for country in country_list:
  code_string = master.loc[master['country'] == country, 'country_code'].to_json()
  code_name = re.search('[A-Za-z]+', code_string).group(0)
  gdp.loc[gdp['Country Name'] == country, 'Country Code'] = code_name
  social.loc[social['country'] == country, 'Country Code'] = code_name
  alc.loc[alc_tob['Country Name'] == country, 'Country Code'] = code_name
  tob.loc[alc_tob['Country Name'] == country, 'Country Code'] = code_name

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
tob.at[1, 'Country Code'] = 'ALB'

In [ ]:
# Load existing geojson point data
with open('countries (1).geojson') as f:
  data = json.load(f)

In [ ]:
# for use with regex in order to add life expectancy data to geojson data
import re

for i in range(0,len(data['features'])):
  country_code = data['features'][i]['properties']['ISO_A3']
  for j in range(2000,2020):
    try:
      continent = regions.loc[regions['alpha-3'] == country_code, 'region'].to_json()
      continent = re.search(':"\D+', continent).group(0)
      continent = continent[2:-2]
      data['features'][i]['properties']['continent'] = continent
    except:
      data['features'][i]['properties']['continent'] = ''
    try:
      sub_region = regions.loc[regions['alpha-3'] == country_code, 'sub-region'].to_json()
      sub_region = re.search(':"\D+', sub_region).group(0)
      sub_region = sub_region[2:-2]
      data['features'][i]['properties']['sub-region'] = sub_region
    except:
      data['features'][i]['properties']['sub-region'] = ''   
    try:
      soc_string = social.loc[social['Country Code'] == country_code, 'system_type'].to_json()
      soc_string = re.search("\[.+\]", soc_string).group(0)
      social_sys = soc_string[1:-1]
      data['features'][i]['properties']['system'] = social_sys
    except:
      data['features'][i]['properties']['system'] = ''
    yr_string = str(j)
    data['features'][i]['properties'][yr_string] = {}
    try:
      expectancy = life_df.loc[life_df['Country Code'] == country_code, f'{yr_string} [YR{yr_string}]']
      expectancy = expectancy.to_json()
      expectancy = re.search("\d\d\.\d\d", expectancy).group(0)
      expectancy = float(expectancy)
      data['features'][i]['properties'][yr_string]['life_expectancy'] = expectancy
    except:
      data['features'][i]['properties'][yr_string]['life_expectancy'] = ''
    try:
      gdp_string = gdp.loc[gdp['Country Code'] == country_code, f'{yr_string}'].to_json()
      gdp_val = float(re.search('\d+\.\d+', gdp_string).group(0))
      data['features'][i]['properties'][yr_string]['gdp'] = gdp_val
    except:
      data['features'][i]['properties'][yr_string]['gdp'] = ''
    try:
      gdp_cap_string = per_cap.loc[per_cap['Country Code'] == country_code, f'{yr_string}'].to_json()
      gdp_cap_val = float(re.search('\d+\.\d+', gdp_cap_string).group(0))
      data['features'][i]['properties'][yr_string]['gdp_pcap'] = gdp_cap_val
    except:
      data['features'][i]['properties'][yr_string]['gdp'] = ''
    try:
      alc_string = alc.loc[alc['Country Code'] == country_code, f'{yr_string}'].to_json()
      alc_level = float(re.search('\d+\.\d+', alc_string).group(0))
      data['features'][i]['properties'][yr_string]['alc_usage'] = alc_level
    except:
      data['features'][i]['properties'][yr_string]['alc_usage'] = ''
    try:
      tob_string = tob.loc[tob['Country Code'] == country_code, f'{yr_string}'].to_json()
      tob_level = float(re.search('\d+\.\d+', tob_string).group(0))
      data['features'][i]['properties'][yr_string]['tob_usage'] = tob_level
    except:
      data['features'][i]['properties'][yr_string]['tob_usage'] = ''   


In [ ]:
# # Write JSON file with new data
with open('region_polygon.geojson', 'w') as f:
   f.write(json.dumps(data))